In [18]:
import numpy as np
import json
import glob
import os
from tqdm import tqdm

In [19]:
# 取得某個階段的所有檔名
def get_filename(root_dir):
    tmp = glob.glob(root_dir + '/*.json')
        
    return tmp

In [12]:
def append_numberOfData(files):
    for i in tqdm(range(len(files))):
        # 讀取原始 json file
        f = open(os.path.join(base_path, files[i]))
        p = json.load(f)
        
        for k in p.keys():
            try:
                keys = list(p[k].keys())
                
                if len(keys) == 13:
                    p[k]['numberOfData'] = 1
                elif len(keys) == 14:
                    p[k]['numberOfData'] = 2
                elif len(keys) == 15:
                    p[k]['numberOfData'] = 3
                elif len(keys) == 16:
                    p[k]['numberOfData'] = 4
                elif len(keys) == 17:
                    p[k]['numberOfData'] = 5
                elif len(keys) == 18:
                    p[k]['numberOfData'] = 6
                elif len(keys) == 19:
                    p[k]['numberOfData'] = 7
                elif len(keys) == 20:
                    p[k]['numberOfData'] = 8
                    
            except:
                pass
        # 刪除原始 json file
        os.remove(os.path.join(base_path, files[i]))
        
        # Write into new json file
        with open(os.path.join(base_path, files[i]), 'w') as file:
            json.dump(p, file)

In [13]:
def modify_json(files):
    for i in tqdm(range(len(files))):
        # 讀取原始 json file
        f = open(os.path.join(base_path, files[i]))
        p = json.load(f)
        
        # 新增 primary key 欄位
        month = int(files[i][:2]) - 12
        month = str(month) if month/10==1 else '0'+str(month)
        version = files[i][-7]
        p['event'] = year[-2:] + str(month) + files[i][2:8] + version
        for k in p.keys():
            try:
                if 'location' in p[k].keys():
                    # 看有幾組資料
                    n_data = len(p[k]['location'])
                    output_dict = {}
                    p[k]['numberOfData'] = n_data
                    
                    # 逐一拿出資料
                    for n in range(n_data):
                        tmp_dict = {}
                        tmp_dict['network'] = p[k]['network'][n]
                        tmp_dict['location'] = p[k]['location'][n]
                        tmp_dict['factor'] = p[k]['factor'][n]
                        tmp_dict['sampling_rate'] = p[k]['sampling_rate'][n]
                        tmp_dict['starttime'] = p[k]['starttime'][n]
                        tmp_dict['endtime'] = p[k]['endtime'][n]
                        tmp_dict['instrument'] = p[k]['instrument'][n]
                        tmp_dict['datatype'] = p[k]['datatype'][n]
                        if n_data > 1:
                            tmp_dict['Z'], tmp_dict['N'], tmp_dict['E'] = p[k]['Z'][n], p[k]['N'][n], p[k]['E'][n]
                        else:
                            tmp_dict['Z'], tmp_dict['N'], tmp_dict['E'] = p[k]['Z'], p[k]['N'], p[k]['E']
                        tmp_dict['pga'], tmp_dict['pgv'] = p[k]['pga'][n], p[k]['pgv'][n]
                        tmp_dict['p_arrival_time'], tmp_dict['s_arrival_time'] = p[k]['p_arrival_time'][n], p[k]['s_arrival_time'][n]
                        tmp_dict['intensity'] = p[k]['intensity'][n]
                        tmp_dict['DataAvailable'] = {}
                        for key in p[k]['DataAvailable'].keys():
                            tmp_dict['DataAvailable'][key] = p[k]['DataAvailable'][key][n]
                            
                        # 新增到依順序建立的新 key
                        output_dict[str(n)] = tmp_dict
                            
                    # 刪除要改掉的 keys
                    del p[k]['network'], p[k]['location'], p[k]['factor'], p[k]['sampling_rate'], p[k]['starttime']
                    del p[k]['endtime'], p[k]['instrument'], p[k]['datatype'], p[k]['Z'], p[k]['N'], p[k]['E']
                    del p[k]['pga'], p[k]['pgv'], p[k]['p_arrival_time'], p[k]['s_arrival_time']
                    del p[k]['intensity'], p[k]['DataAvailable']
                            
                    # 把改好的加進原始資料中
                    for modify_k in output_dict.keys():
                        p[k][modify_k] = output_dict[modify_k]
                        
            except Exception as e:
                #print(e)
                pass
            
        # 刪除原始 json file
        os.remove(os.path.join(base_path, files[i]))
        
        # Write into new json file
        with open(os.path.join(base_path, files[i]), 'w') as file:
            json.dump(p, file)
        

In [15]:
# target: 2020, 2019, 2018, 2017, 2016, 2015. 2014
#all_year = ['2020', '2019', '2018', '2017', '2016', '2015', '2014']
all_year = ['2019']
for year in all_year:
    sub_fname = year[2:]
    base_path = os.path.join('/mnt/nas6/CWBSN', year)
    files = os.listdir(base_path)
    
    modify_json(files)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 564/564 [3:06:13<00:00, 19.81s/it]


## modify primary key

In [46]:
# finish: 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
year = '2013'
root = "/mnt/nas6/CWBSN/"+year
files = get_filename(root)

In [48]:
for file in tqdm(files):
    try:
        f = open(os.path.join(root, file), 'r')
        p = json.load(f)

        month = int(file[-16:-14])-12
        month = str(month) if month//10==1 else '0'+str(month)
        pk = year[-2:]+month+file[-14:-8]+file[-7]
        p['event'] = pk

        os.remove(os.path.join(root, file))
        with open(os.path.join(root, file), 'w') as f:
            json.dump(p, f)
    except:
        with open('/mnt/nas6/CWBSN/modifyPK_2013.txt', 'w') as f:
            f.write(file+'\n')
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1168/1168 [2:16:13<00:00,  7.00s/it]


In [44]:
f = open(os.path.join(root, files[1]), 'r')
p = json.load(f)

In [45]:
files[0]

'/mnt/nas6/CWBSN/2013/15201638(0).json'